# Дополнительный проект. SQL

## Задачи

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так будут исключены из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг (учесть только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

In [20]:
import pandas as pd
from sqlalchemy import create_engine

In [21]:
db_config = {
  'user': 'praktikum_student',
  'pwd': 'Sdf4$2;d-d30pp',
  'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
  'port': 6432,
  'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
  db_config['user'],
  db_config['pwd'],
  db_config['host'],
  db_config['port'],
  db_config['db']
)


engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [22]:
def pd_select(sql):
    return pd.io.sql.read_sql(sql, con=engine)

## Исследование таблиц

### Таблица `books` - данные о книгах

In [23]:
pd_select('SELECT * FROM books LIMIT 5;')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


- `book_id` — идентификатор книги
- `author_id` — идентификатор автора
- `title` — название книги
- `num_pages` — количество страниц
- `publication_date` — дата публикации книги
- `publisher_id` — идентификатор издателя

In [24]:
pd_select('SELECT * FROM books').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


### Таблица `authors` - данные об авторах

In [25]:
pd_select('SELECT * FROM authors LIMIT 5;')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


- `author_id` — идентификатор автора
- `author` — имя автора

In [26]:
pd_select('SELECT * FROM authors').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


### Таблица `publishers` - данные об издательствах

In [27]:
pd_select('SELECT * FROM publishers LIMIT 5;')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- `publisher_id` — идентификатор издательства
- `publisher` — название издательства

In [28]:
pd_select('SELECT * FROM publishers').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Таблица `ratings` - данные о пользовательских оценках книг

In [29]:
pd_select('SELECT * FROM ratings LIMIT 5;')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


- `rating_id` — идентификатор оценки
- `book_id` — идентификатор книги
- `username` — имя пользователя, оставившего оценку
- `rating` — оценка книги

In [30]:
pd_select('SELECT * FROM ratings').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### Таблица `reviews` - данные о пользовательских обзорах на книги

In [31]:
pd_select('SELECT * FROM reviews LIMIT 5;')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- `review_id` — идентификатор обзора
- `book_id` — идентификатор книги
- `username` — имя пользователя, написавшего обзор
- `text` — текст обзора

In [32]:
pd_select('SELECT * FROM reviews').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


## Задачи

### Количество книг, вышедших после 1 января 2000 года

In [33]:
sql = '''
SELECT
  count(1)
FROM books
WHERE
  publication_date > '01.01.2000'::date
'''

pd_select(sql)

,count
0,819


После 1 января 2000 года (не включительно) вышло **819** книг.

### Количество обзоров и средняя оценка для каждой книги

In [34]:
sql = '''
WITH w_reviews as (
  SELECT
    rv.book_id as book_id,
    count(rv.book_id) as cnt
  FROM reviews rv
  GROUP BY rv.book_id
), w_ratings as (
  SELECT
    rt.book_id as book_id,
    count(rt.book_id) as cnt,
    avg(rt.rating) as rating
  FROM ratings rt
  GROUP BY rt.book_id
)
SELECT
  b.book_id as book_id,
  b.author_id as author_id,
  b.title as title,
  COALESCE(w_reviews.cnt, 0) as num_reviews,
  COALESCE(w_ratings.cnt, 0) as num_ratings,
  w_ratings.rating as avg_rating
FROM books b
LEFT JOIN w_reviews ON (
  w_reviews.book_id = b.book_id
)
LEFT JOIN w_ratings ON (
  w_ratings.book_id = b.book_id
)
ORDER BY num_reviews DESC, avg_rating DESC
LIMIT 5
'''

pd_select(sql)

,book_id,author_id,title,num_reviews,num_ratings,avg_rating
0,948,554,Twilight (Twilight #1),7,160,3.662500
1,302,236,Harry Potter and the Prisoner of Azkaban (Harr...,6,82,4.414634
2,299,236,Harry Potter and the Chamber of Secrets (Harry...,6,80,4.287500
3,656,402,The Book Thief,6,53,4.264151
4,734,278,The Glass Castle,6,29,4.206897


Наибольшее количество обзоров имеют книги:
- Twilight (Twilight #1) - 7 обзоров
- Harry Potter and the Prisoner of Azkaban - 6 обзоров
- Harry Potter and the Chamber of Secrets - 6 обзоров

Наибольшее количество оценок получили книги:
- Twilight (Twilight #1) - 160 оценок
- The Hobbit or There and Back Again - 88 оценок
- The Catcher in the Rye - 86 оценок

Наиболее высокую оценку получили книги:
- And Then There Were None - 4.45 средняя оценка (22 оценки)
- Gone with the Wind - 4.43 средняя оценка (35 оценок)
- Harry Potter and the Prisoner of Azkaban - 4.41 средняя оценка (82 оценки)

*Учитывались только книги, получившие не менее 20 оценок*

Все книги получили как минимум одну оценку.

Для 5 книг не было получено ни одного обзора:
- The Cat in the Hat and Other Dr. Seuss Favorites
- Leonardo's Notebooks
- Disney's Beauty and the Beast
- Essential Tales and Poems
- Anne Rice's The Vampire Lestat: A Graphic Novel
- The Natural Way to Draw	

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [35]:
sql = '''
SELECT
  b.publisher_id as publisher_id,
  pb.publisher as publisher,
  count(b.book_id) as num_books
FROM books b
LEFT JOIN publishers pb ON (
  pb.publisher_id = b.publisher_id
)
WHERE b.num_pages > 50
GROUP BY b.publisher_id, pb.publisher
ORDER BY num_books DESC
LIMIT 3
'''

pd_select(sql)

,publisher_id,publisher,num_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25


Издательство **Penguin Books** выпустило наибольшее число книг толще 50 страниц.

### Автор с самой высокой средней оценкой книг

Учитываются только книги с 50 и более оценками.

In [36]:
sql = '''
WITH w_ratings as (
  SELECT
    rt.book_id as book_id,
    count(rt.book_id) as cnt
  FROM ratings rt
  GROUP BY rt.book_id
)
SELECT
  b.author_id as author_id,
  a.author as author,
  avg(rt.rating) as avg_rating
FROM ratings rt
LEFT JOIN w_ratings ON (
  w_ratings.book_id = rt.book_id
)
LEFT JOIN books b ON (
  b.book_id = rt.book_id
)
LEFT JOIN authors a ON (
  a.author_id = b.author_id
)
WHERE w_ratings.cnt >= 50
GROUP BY b.author_id, a.author
ORDER BY avg_rating DESC
LIMIT 3
'''

pd_select(sql)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914


Автор с самой высокой средней оценкой книг - **J.K. Rowling/Mary GrandPré**.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [37]:
sql = '''
WITH w_users as (
  SELECT
    rt.username as username
  FROM ratings rt
  GROUP BY rt.username
  HAVING count(rt.username) > 50
), w_reviews as (
  SELECT
    rv.username as username,
    count(rv.username) as cnt
  FROM reviews rv
  WHERE rv.username in (SELECT username FROM w_users)
  GROUP BY rv.username
)
SELECT
  avg(w_reviews.cnt)
FROM w_reviews
'''

pd_select(sql)

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили более 50 оценок, равно **24**.